# MUM 2023-24 Optymalizatory adaptacyjne

<img style="float: right;" src="ml_figures/conv_nn.png" width=450>

* wartości bezwzględne gradientów mogą się bardzo różnić między warstwami dużych sieci neuronowych
  * globalne $\gamma_t$ może nie działać poprawnie
  * śledzić i uaktualniać współczynnik uczenia dla każdego parametru
  * algorytmy adaptują się do _wariancji_ wag albo lokalnej krzywizny problemu (powierzchni błędu)

##### Uwaga

1. W oryginalnych pracach dotyczących niektórych z poniższych metod pojawiają się matematyczne "akrobacje", które polegają na przykład na definiowaniu "pierwiastka ze zdiagonalizowanej macierzy produktu zewnętrznego historii gradientów". Proszę absolutnie nie myśleć w ten sposób o tych optimizerach!

2. W tym notebooku wszystkie operacje na wektorach są zdefiniowane _element-wise_, zgodnie z regułami pakietu numpy. Na przykład:
    * kwadrat wektora to wektor kwadratów elementów (numpy.square)
    * pierwiastek z wektora to wektor pierwiastków z elementów (numpy.sqrt)
    * suma, różnica, iloraz, iloczyn - analogicznie
    * suma wektora i liczby oznacza dodanie tej liczby do każdej współrzędnej

## Adagrad
<img style="float: right;" src="ml_figures/opt2.gif" width=450>

* $\gamma_t$ - współczynnik uczenia (learning rate), typowo od 0.001 do 0.01
* $\epsilon$ - zapobiega dzieleniu przez zero (zwykle $10^{-8}$)
* $h_t$ - wektor sumujący kwadraty gradientów do czasu $t$, wymiar taki sam jak $w$; $h_t(0)=0$
$$\begin{align}
h_{t+1} &= h_t + \nabla L(w_t)^2\\
w_{t+1}&=w_t - \gamma_t \dfrac{\nabla L(w_t)}{\sqrt{h_{t+1} + \epsilon}}
\end{align}$$
1. dzielenie normalizuje gradient
2. normalizacja oddzielnie dla:
    * każdej współrzędnej gradientu
      * czyli dla każdej współrzędnej $w$ - osobno dla każdego parametru modelu
3. $h_t$ jest sumą kwadratów, więc trzeba w mianowniku wziąć pierwiastek - bez pierwiastka działa słabo
4. __akumulacja__ kwadratów gradientów
    * $h$ to suma, a nie średnia
    * gdyby gradienty były stałe, mianownik rósłby proporcjonalnie do $\sqrt{t}$
    * w praktyce maleje $\Delta w$ i model może __przestać się uczyć__

## RMSProp (Hinton)
<img style="float: right;" src="ml_figures/opt2.gif" width=450>

* normalizacja przez pierwiastek wartości oczekiwanej gradientu
* $\gamma$ - learning rate (typowe wartości: od 0.001 do 0.01)
* $\alpha$ - współczynnik średniej kroczącej (zazwyczaj: 0.9)
* $\epsilon$ - zapobiega dzieleniu przez zero (zazwyczaj: $10^{-8})$
* $v_t$ - wektor średniej kroczącej kwadratów gradientów do czasu $t$, wymiar taki sam jak $w$
  * $v_t$ jest estymacją drugiego momentu
  * większość algorytmów używa średnich kroczących dla estymacji szumu, który zmienia się w czasie
$$\begin{align}
v_{t+1} &= \overbrace{\alpha v_{t} + \overbrace{(1-\alpha)\nabla L(w_t)^2}^{\textrm{kwadrat po elementach (element-wise)}}}^{\textrm{wykładnicza średnia krocząca}}\\
w_{t+1}&=w_t - \gamma\frac{\nabla L(w_t)}{\sqrt{v_{t+1}} + \epsilon}
\end{align}$$
  * $\epsilon$ zabezpiecza przed dzieleniem przez zero

## Adadelta
<img style="float: right;" src="ml_figures/opt2.gif" width=450>

* $\alpha$ - współczynnik średniej kroczącej (zazwyczaj: 0.95)
* $\epsilon$ - zapobiega dzieleniu przez zero, umożliwia rozpoczęcie uczenia (zazwyczaj: od $10^{-6}$ do $10^{-2}$)

* $v_t$ - wektor średniej kroczącej kwadratów gradientów do czasu $t$ element po elemencie
* $d_t$ - wektor średniej kroczącej $\Delta{}w$ do czasu $t$ element po elemencie
$$\begin{align}
v_{t+1} &= \alpha{}v_t + (1-\alpha)(\nabla L(w_t)^2\\
w_{t+1} &=w_t - \sqrt{d_t + \epsilon}\; \dfrac{\nabla L(w_t)}{\sqrt{v_{t+1} + \epsilon}}\\
d_{t+1} &= \alpha d_t + (1-\alpha)(w_{t+1} - w_{t})^2
\end{align}$$
1. rozszerzenie RMSProp (ale zaproponowane niezależnie jako poprawka Adagrad)
2. zastąpienie learning rate przez __wektor__
    * learning rate __proporcjonalny do__ średniego $\Delta{}w$
    * upodobnienie szybkości poprawek do poprawek
    * __eliminacja__ stałej uczenia z parametrów
3. ważna rola parametru $\epsilon$
    * nie tylko zapobiega dzieleniu przez zero
    * umożliwia rozpoczęcie uczenia - $d_1$ jest większe od zera
    * $\sqrt{\epsilon}$ wyznacza __dolne ograniczenie__ $\sqrt{d_t +\epsilon}$, a stąd uczenie nie zatrzymuje się

## [Adam: a method for stochastic optimization, D.P. Kingma, J. Ba, 2015](https://arxiv.org/pdf/1412.6980.pdf)
* rodzaj RMSprop z elementem momentum
* $\gamma$ - learning rate (zazwyczaj: 0.001)
* $\beta_1$ - współczynnik średniej kroczącej pierwszego momentu (zazwyczaj: 0.9)
* $\beta_2$ - współczynnik średniej kroczącej pierwszego momentu (zazwyczaj: 0.999)
* $\epsilon$ - zapobiega dzieleniu przez zero (zazwyczaj: $10^{-8}$)
* $m_t$ - wektor średniej kroczącej pierwszego momentu gradientu do czasu $t$
* $v_t$ - wektor średniej kroczącej drugiego momentu gradientu do czasu $t$
$$\begin{align}
m_{t+1} &= \beta_1 m_t + (1-\beta_1)(\nabla L(w_t)\\
v_{t+1} &= \beta_2 v_t + (1-\beta_2)(\nabla L(w_t))^2\\
\widehat m &= \dfrac{m_{t+1}}{1-\beta_1^{(t+1)}}\\
\widehat v &= \dfrac{v_{t+1}}{1-\beta_2^{(t+1)}}\\
w_{t+1} &= w_t - \gamma\dfrac{\widehat m}{\sqrt{\widehat v} + \epsilon}
\end{align}$$
__Uwaga__ $\beta^{t+1}$ to "$\beta$ do potęgi $t+1$", a nie $\beta$ w czasie $t+1$
albo skrótowo w stabilnej wersji, która jest szybko osiągana
$$\begin{align}
m_{t+1} &= \beta_1 m_t + (1-\beta_1)(\nabla L(w_t)\\
v_{t+1} &= \beta_2 v_t + (1-\beta_2)(\nabla L(w_t))^2\\
w_{t+1} &= w_t - \gamma\dfrac{m_t}{\sqrt{v_{t+1}} + \epsilon}
\end{align}$$
  * do aktualizacji wag używany jest wektor estymujący $m_t$ przez średnią kroczącą
    * średnia krocząca __zbiasowana__ w kierunku zera
      * "uśrednienia" wprowadzają poprawkę: im później, tym poprawka mniejsza
  * __gradient__ adaptowany
    * krocząca średnia gradientów (pierwszy moment) - tłumione oscylacje
    * krocząca średnia kwadratów gradientów (drugi moment) - normalizacja gradientów
  * eksponencjalna średnia krocząca estymująca momentum jest równoważna standardowej postaci przy przeskalowaniu

* Adam jest zwykle znacznie lepszy od SGD dla problemów, które są źle uwarunkowane, a zwykle są 😞
  * często jednak Adam nie zbiega dla prostych problemów!
* Adam ma przewagę nad RMSprop ze względu na uwzględnienie momentum
* Adam nie jest dobrze zrozumiały teoretycznie
* dla wielu problemów związanych z obrazami, na przykład ImageNet, daje gorsze wyniki generalizacji
* wymaga więcej pamięci niż SGD
* ma dwa parametry momentum, skąd może być potrzebne trochę tunowania
  * zwykle warto wypróbować SGD z momentu oraz Adama z szeregiem różnych parametrów by wybrać

[Wizualizacja uczenia dla wielu algorytmów [deeplearning.ai]](https://www.deeplearning.ai/ai-notes/optimization/index.html)

## I wiele innych

* AdaMax, Nadam, AMSGrad, ...
* algorytmy uczenia mogą być specyficzne dla konkretnych architektur i zadań
  * na przykład uczenie sprzętowych architektur wprost
    * bardzo rzadko — nie są one dostosowane do uczenia
    * implementacja uczenia i inferencji bardzo się różnią od siebie
    * znacznie prościej jest uczyć symulowaną architekturą a znalezione wagi skopiować
* uczenie NN nie musi być gradientowe
  * gradientowe wymaga by funkcja kosztu była różniczkowalna
    * z tego powodu nie możemy uczyć wprost by celem była jak najlepsza klasyfikacja
* alternatywą jest uczenie _perturbacyjne_
  * dla aktualnego przykładu wagi są _perturbowane_ o małe $-\epsilon$ i $+\epsilon$
  * wybierana jest poprawka, która daje lepszy wynik
* wiele zaawansowanych metod wykorzystywanych w płytszych sieciach neuronowych nie jest wykorzystywanych dla sieci głębokich
  * metody przybliżania drugiej pochodnej nie dają wiele w porównaniu do metod będących uogólnieniami metody momentum
  * wykorzystanie macierzy Hesjanu jest bardzo trudne ze względu na dużą liczbę parametrów

## niektóre cechy pozwalające na pierwszą decyzję o wyborze algorytmu [na podstawie deeplearning.ai]
[Wizualizacja uczenia dla wielu algorytmów [deeplearning.ai]](https://www.deeplearning.ai/ai-notes/optimization/index.html)
* **SGD**
  * proste urównoleglenie, jednak powolne gdy pamięć GPU jest niewystarczająca
  * SGD szybciej zbiega na dużych zbiorach danych ze względu na częstszą aktualizację
  * lepsza aproksymacja gradientu ze względu na wykorzystanie redundancji danych
  * najmniej użytej pamięci dla ustalonej wielkości batcha
* **momentum**
  * przyspiesza uczenie, wykorzystując minimalną modyfikację algorytmu
  * więcej pamięci na batch niż SGD, ale sporo mniej niż RMSprop czy Adam
* **RMSprop**
  * adaptacyjne podejście zapobiega zbyt szybkiemu zanikaniu czy wybuchowi hiperparametru uczenia
  * utrzymuje prędkości uczenia odpowiednie dla każdego parametru modelu (wagi sieci)
  * więcej pamięci niż momentum, ale mniej niż Adam
* **Adam**
  * hiperparametry algorytmu mogą być zwykle ustawione na wartości domyślne i nie potrzebują dodatkowego dopasowania
    * a jest ich wiele — stała uczenia, eksponencjalna stała zanikania dla momentum, etc.
  * realizuje formę statystycznego podejścia **wyżarzania** (ang. annealing) z adaptacyjnymi krokami uczenia
  * zużywa najwięcej pamięci na batch
  * jest zwykle domyślnym optymalizatorem
    * może być ważne przy porównywaniu rozwiązań